In [31]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string
import csv
import sys
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import sklearn.metrics
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

In [2]:
def parseLabeledData(path):
    file=open(path, 'r')
    dataList = []
    for line in csv.reader(file):        
        if len(line) >= 5:
            dataList.append(
                {"asin":line[0], 
                 "question":line[1],
                 "review":line[2],
                 "answer":line[3],
                 "label":line[4]}
            )     
    return dataList
        

print("Reading labeled data...")
data = parseLabeledData("C:/Users/Moi/Downloads/out.csv")
queries = [d['question'] for d in data]
answers = [d['answer'] for d in data]
reviews = [d['review'] for d in data]
labels = [d['label'] for d in data]
print("done")

Reading labeled data...
done


In [3]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

print("Reading all reviews & all questions...")
allReviews = parseAllReviews("C:/Users/Moi/Downloads/reviews.json")
allQuestions = parseAllQueries("C:/Users/Moi/Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = []
for entry in allReviews.values():
    for review in entry:
        docSet.append(review["reviewText"])

for entry in allQuestions.values():
    for question in entry:
        docSet.append(question["question"])

docLen = [len(d.split()) for d in docSet]
avgdl = sum(docLen) / len(docLen)

print("done")

Reading all reviews & all questions...
done


In [48]:
def parse(path):
  file = open(path, 'r')
  for l in file:
    yield eval(l)

print("Reading data...")
products = list(parse("C:/Users/Moi/Downloads/reviews.json"))
# http://jmcauley.ucsd.edu/data/amazon/qa/qa_Tools_and_Home_Improvement.json.gz
allQuestionsText = list(parse("C:/Users/Moi/Downloads/qa.json"))
allReviewsText = defaultdict(lambda: [])
for product in products:
    allReviewsText[product["asin"]].append(product["reviewText"])
print("done")

Reading data...
done


In [4]:
def countAllWords():
    allWords = defaultdict(int)
    englishStopWords = stopwords.words('english')
    for r in allReviews.values():
        for review in r:
            review = review["reviewText"]
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            question = question["question"]
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1
    
    
    return allWords

allWords = countAllWords()

In [5]:
commonWords = sorted(allWords, key=lambda x: -allWords[x])[:1000]

In [6]:
# idfDict = defaultdict(float)
#for word in commonWords:
#    count = 0         
#    for doc in docSet:
#        if word in doc.lower():
#            count += 1
#    idfScore = math.log(len(docSet)/(count+1))
#     
#    idfDict[word] = idfScore


In [7]:
# @param a word whose frequency in the document we are calculating
# @param document a string of a review or a question
# @return the frequency of term in document div length of document

def tf(term, document):
    count = collections.defaultdict(int)
    exclude = set(string.punctuation)
    document = ''.join(ch for ch in document if ch not in exclude)
    for word in document.split():
        count[word] += 1

    return count[term]/(len(document.split()) + 1)

In [8]:
idfDict = defaultdict(float)

def idf(term):
    term = term.lower()
    if (term in idfDict):
        return idfDict[term]

    count = 0
    for doc in docSet:
        #exclude = set(string.punctuation)
        #doc = ''.join(ch for ch in doc if ch not in exclude)
        if term in doc.lower():
            count += 1
        
    idfScore = math.log(1 + len(docSet) / (count+1))
    idfDict[term] = idfScore
    return idfScore

In [9]:
okapidict = {}

def OkapiBM25(review, question, k1, b):
    if ((review, question, k1, b) in okapidict):
        return okapidict[review, question, k1, b]
    
    question = question.lower()
    question = ''.join([c for c in question if not (c in string.punctuation)])
    
    score = 0
    for q in question.split():
        num = tf(q, review) * (k1 + 1)
        den = tf(q, review) + k1 * (1 - b + b*len(review.split()) / avgdl) 
        score += idf(q) * num / den
        
    #print(score, review, question)
    okapidict[review, question, k1, b] = score
    return score

In [10]:
tfidfdict = {}

def tfidf(document):
    if (document in tfidfdict):
        return tfidfdict[document]
    
    doc = document.lower()
    doc = ''.join([c for c in doc if not (c in string.punctuation)])
        
    feat = collections.defaultdict(int)
    for term in doc.split():
        tfscore = tf(term, doc)
        idfscore = idf(term)
        feat[term] = tfscore * idfscore
        
    tfidfdict[document] = feat
    return feat

In [11]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

In [12]:
def numCommonWords(review, question):
    review = review.lower()
    review = ''.join([c for c in review if not (c in string.punctuation)])
    
    question = question.lower()
    question = ''.join([c for c in question if not (c in string.punctuation)])
    
    filtered_words = [word for word in question.split() if word not in stopwords.words('english')]
    words = set(filtered_words)
    
    num = 0
    for word in words:        
        if word in review:
            num += 1
  
    #print(num)
    return num

#numCommonWords("This is a red and blue review about a car", "I am, a BLUE and yellow quESTION about a car!")

In [13]:
def lengthDiff(review, question):
    return abs(len(review.split()) - len(question.split()))

In [14]:
# queryFeat is a feature vector for the query and reviewFeat is the feature vector for the review
def cosineSimilarity(queryFeat, reviewFeat):
    # Find the words the 2 dictionaries have in common
    querySet = set(queryFeat.keys())
    reviewSet = set(reviewFeat.keys())
    allWords = querySet.union(reviewSet)
    
    # Find the cosine similarity
    numerator = 0
    mag1 = 0
    mag2 = 0
    for word in allWords:
        numerator = numerator + queryFeat[word] * reviewFeat[word]
        mag1 = mag1 + queryFeat[word]**2
        mag2 = mag2 + reviewFeat[word]**2
    if mag1 > 0 and mag2 > 0:
        return (numerator/((mag1*mag2)**0.5))
    else:
        return -1

In [15]:
def uniqueWords(review, question, num):
    exclude = set(string.punctuation)
    question = ''.join(ch for ch in question if ch not in exclude)
    review = ''.join(ch for ch in review if ch not in exclude)
    review = review.lower()
    
    qFreq = {word : allWords[word] for word in question.lower().split()}
    
    topUnique = [word for word in sorted(qFreq, key=lambda x: qFreq[x]) if allWords[word] != 0]
    
    if num <= len(topUnique):
        topUnique = topUnique[:num]
    else:
        topUnique += [''] * (num - len(topUnique))
    
    #print(qFreq)
    #print(topUnique)
    
    feat = []
    for word in topUnique:
        #feat.append(review.split().count(word))
        feat.append(1 if word in review and word != '' else 0)
        
    return feat

#uniqueWords("The color of this item is RED red red Great", "Hello, this color ReD? I think great", 10)

In [16]:
def constrain(elems, point):    
    for elem in elems:
        if (elem > point): yield 1
        else: yield 0
        #if elem > 1: yield 1
        #elif elem < 0: yield 0
        #else: yield elem

In [17]:
def normalize(featList):
    
    max = 0
    min = float('inf')
    for feat in featList:
        if feat > max: max = feat
        if feat < min: min = feat        
    
    for i in range(0,len(featList)-1):
        if (max - min) == 0: 
            max = 1
            min = 0
        featList[i] = (featList[i] - min) / (max - min)

    return featList

In [18]:
def feature_tfidf(review, question):
    feat = [1]
    feat.append(cosineSimilarity(tfidf(review), tfidf(question)))
    return feat

In [19]:
def feature_okapi(review, question):
    feat = [1] 
    feat.append(OkapiBM25(review, question, 1.5, 0.75))
    return feat

In [20]:
def feature(review, question, length):
    feat = [1]
    
    #number of Common Words
    #difference in length
    #length of review
    #length of question
    feat.append(numCommonWords(review, question))
    #feat.append(lengthDiff(review,question))
    #feat.append(len(review.split()))
    #feat.append(len(question.split()))
    cosine = cosineSimilarity(tfidf(review), tfidf(question))
    feat.append(cosine)
    feat.append(OkapiBM25(review, question, 1.5, 0.75))
    feat = feat + uniqueWords(review, question, length)
    
    return feat

In [74]:
def test(y, y_hat):
    #print(sklearn.metrics.r2_score(y, y_hat))
    
    
    accuracy = sklearn.metrics.accuracy_score(y, y_hat)
    precision = sklearn.metrics.precision_score(y, y_hat)
    recall = sklearn.metrics.recall_score(y, y_hat)
    
    return "{0:.2f}, {1:.2f}, {2:.2f}".format(accuracy, precision, recall)

def train(X, y, breakpoint=0.15):
    random.seed(4410)
    
    for j in range(0,len(X[0])):
        featList = []
        for i in range(0,len(X)-1):
            featList.append(X[i][j])
        featList = normalize(featList)
        for i in range(0,len(X)-1):
            X[i][j] = featList[i]

    lr.fit(X, y)
    
    return lr

lr = LogisticRegression()
def pipeline(X, y, breakpoint=0.15):
    random.seed(4410)
    
    for j in range(0,len(X[0])):
        featList = []
        for i in range(0,len(X)-1):
            featList.append(X[i][j])
        featList = normalize(featList)
        for i in range(0,len(X)-1):
            X[i][j] = featList[i]
    
    keys = list(range(1, len(labels)))
    points = dict(zip(keys, zip(X, y)))
    random.shuffle(keys)
    X_rand = [points[key][0] for key in keys]
    y_rand = [points[key][1] for key in keys]
    
    X_train = X[:len(X_rand)*2//3]
    y_train = y[:len(y_rand)*2//3]
    
    X_test = X[len(X_rand)*2//3:]
    y_test = y[len(y_rand)*2//3:]
       
    lr.fit(X_train, y_train)
    y_hatTrain = lr.predict(X_train)
    y_hatTest = lr.predict(X_test)
    
    #print(y_hatTrain)
    
    
    y_confidenceTrain = lr.decision_function(X_train)
    y_confidenceTest = lr.decision_function(X_test)
    
    #print(y_confidenceTrain)
    

    
    return (y_confidenceTrain, y_confidenceTest, y_train, y_test)

    

for length in range(0, 3):
    print("Our model looking at", length, "unique words:")
    
    X = [feature(d["review"], d["question"], length) for d in data]
    y = [1 if l == "Y" else 0 for l in labels]
    y_hatTrain, y_hatTest, y_train, y_test = pipeline(X, y)
    
    for cutoff in range(-50, 0):
        y_hatTrain_c = list(constrain(y_hatTrain, cutoff / 20))
        y_hatTest_c = list(constrain(y_hatTest, cutoff / 20))
        train_string = test(y_train, y_hatTrain_c)
        test_string = test(y_test, y_hatTest_c)

        print(cutoff / 20, ":\t", train_string, "\t", test_string)
    print()


Our model looking at 0 unique words:
-2.5 :	 0.14, 0.14, 1.00 	 0.24, 0.23, 1.00
-2.45 :	 0.34, 0.17, 1.00 	 0.42, 0.28, 1.00
-2.4 :	 0.39, 0.18, 1.00 	 0.48, 0.30, 0.98
-2.35 :	 0.43, 0.19, 0.99 	 0.51, 0.32, 0.98
-2.3 :	 0.46, 0.20, 0.99 	 0.52, 0.32, 0.98
-2.25 :	 0.46, 0.20, 0.99 	 0.52, 0.32, 0.98
-2.2 :	 0.46, 0.20, 0.99 	 0.52, 0.32, 0.98
-2.15 :	 0.53, 0.22, 0.97 	 0.60, 0.36, 0.98
-2.1 :	 0.57, 0.23, 0.91 	 0.65, 0.39, 0.95
-2.05 :	 0.62, 0.25, 0.88 	 0.69, 0.42, 0.90
-2.0 :	 0.65, 0.25, 0.78 	 0.70, 0.42, 0.86
-1.95 :	 0.66, 0.25, 0.75 	 0.71, 0.43, 0.86
-1.9 :	 0.67, 0.26, 0.75 	 0.72, 0.44, 0.86
-1.85 :	 0.69, 0.26, 0.71 	 0.72, 0.45, 0.84
-1.8 :	 0.72, 0.28, 0.68 	 0.76, 0.48, 0.81
-1.75 :	 0.73, 0.27, 0.59 	 0.76, 0.49, 0.79
-1.7 :	 0.74, 0.27, 0.57 	 0.78, 0.51, 0.72
-1.65 :	 0.75, 0.29, 0.57 	 0.77, 0.49, 0.66
-1.6 :	 0.76, 0.29, 0.55 	 0.76, 0.47, 0.57
-1.55 :	 0.76, 0.30, 0.52 	 0.77, 0.50, 0.53
-1.5 :	 0.78, 0.30, 0.46 	 0.75, 0.44, 0.40
-1.45 :	 0.80, 0.33, 0.43 	 0

In [84]:
def questionAndReviews(qa, reviews):
    X = []
    dictionary = []
    
    count = 0
    
    for question in qa:
        
    #while numReviews < 70:
        if (question['questionType'] != "yes/no"): continue
        if (question['answerType'] == '?'): continue
        #if (question['asin'] in questionSet): continue
        questionText = question['question']
        asin = question['asin']
        answer = question['answerType']
        numReviews = len(reviews[asin])
        if (numReviews < 70): continue
            
                    
        count += 1
        
        if (count > 500): break
    
        # reviewsSet is list of all review text for this product
        reviewsSet = reviews[asin]
        
        questionFeat = []
        questionDictionary = []
        for review in reviewsSet:
            questionFeat += [feature(review, questionText, 2)]
            questionDictionary += [(questionText, review, asin, answer)]
        
        X += [questionFeat]
        dictionary += [questionDictionary]

    return (X, dictionary)

In [85]:
(allFeature, featureMap) = questionAndReviews(allQuestionsText,allReviewsText)

In [81]:
for features, kvpairs in zip(allFeature, featureMap):
    y_hat = lr.decision_function(features)
    
    questionText = kvpairs[0][0]
    
    maxnum = 0
    maxi = 0
    for i in range(0, len(y_hat)):
        num = y_hat[i]
        
        if (num > maxnum): 
            maxnum = num
            maxi = i
    
    reviewText = kvpairs[maxi][1]
    
    print(questionText, "\n", reviewText, "\n")
    
    

Would you use this for a framing/finish nail gun? 
 I use this hose primarily for my nail guns.  It is nice and heavy so it lays flat when in use.  It also winds up well unlike a pvc or plastic hose.  The ends are installed well.  This product is clearly superior to Chinese knock-offs that you may find at a big-box store.  Don't try to save $4; buy this one! 

I want a soft air hose so it's easy roll up and work with. My air hose is hard and hard to work with. Is this one easy to work with? Thanks. 
 This hose is both sturdy and consistent. Since it is rubber it gets softer when it is up on a hot roof in the summer and it gets stiffer when it is used in freezing weather. But it resists tangling as long as I take the time to get the twist out as I unroll it. I have had no problem with soft spots, blow outs nor leaks. It has resisted the little bit of oil I've gotten on it. The fittings are solidly placed, male on one end and female on the other so I can put the ends together and keep th

In [86]:
with open("C:/Users/Moi/Downloads/highestReviewData.csv", 'w') as the_file:

    for features, kvpairs in zip(allFeature, featureMap):
        
        y_hat = lr.decision_function(features)
    
        questionText = kvpairs[0][0]

        maxnum = 0
        maxi = 0
        for i in range(0, len(y_hat)):
            num = y_hat[i]

            if (num > maxnum): 
                maxnum = num
                maxi = i

        reviewText = kvpairs[maxi][1]
        
        asin = kvpairs[0][2]
        answer = kvpairs[0][3]
        
        the_file.write(asin)
        the_file.write(',')
        the_file.write(questionText.replace(',', ''))
        the_file.write(',')
        the_file.write(reviewText.replace(',', ''))
        the_file.write(',')
        the_file.write(answer.replace(',', ''))
        the_file.write('\n')